### PyEmma Featurizer Support

In [1]:
import openpathsampling as ps
import openpathsampling.storage as st

In [2]:
#! lazy
import pyemma.coordinates as coor

In [3]:
storage = st.Storage('trajectory.nc', mode='a')

Import a PyEmma Coordinates Module

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [4]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [5]:
cv = ps.collectivevariable.CV_PyEMMA_Featurizer('pyemma', pyemma_generator, topology=storage.template.topology)

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [6]:
cv(storage.trajectories[0]);

Let's save it to the storage

In [7]:
cv.save(storage)

(u'snapshots', u'snapshots_cv_10_values_dim_1') None


and apply the featurizer to a trajectory

In [8]:
cv(storage.trajectories[2]);

Sync to make sure the cache is written to the netCDF file.

In [9]:
storage.sync_all()

In [10]:
cv(storage.snapshots.all());

In [11]:
storage.cvs.sync()

In [12]:
storage.sync_all()

In [13]:
py_cv = storage.cvs['pyemma']

In [14]:
nc_var = storage.variables['snapshots_cv_%d_values' % storage.idx(py_cv)]

In [15]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [16]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [17]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "psi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[6, 8, 14, 16]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "phi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[4, 6, 8, 14]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 0}, "store_cache": true, "name": "StateA"}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 25}, "store_cache": true, "name": "StateB"}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 26}, "store_cache": true, "name": "StateX"}}'
 u'{"_cls": "CV_PyEMMA_F

In [18]:
py_emma_feat = storage.vars['cvs_json'][5]

In [19]:
erg = py_emma_feat(storage.snapshots);

In [20]:
#! lazy
print erg[0::10,1]

[ 4.05474138  4.02489281  4.03605986  4.01896954  4.00349855  4.13938951
  4.06451654  3.99037433  3.92382956  3.99639368  4.17125559  4.03728819
  3.98762965  4.07114124  3.89337707  4.01789045  3.94613504  4.16219521
  3.98163962  3.84772992  3.93644977  4.09034443  4.00903463  4.0276351
  3.89627552  3.98913574  4.12395716  3.89439797  3.93218327  4.12557936
  4.09141016  3.91008472  4.05836582  3.91347909  4.0663991   4.03691959
  3.89385867  4.04961586  4.085145    4.06395721  4.07543802  4.36958742
  4.04664898]
